In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
from openai import OpenAI
import requests
import textwrap

# 這裡填你的 ngrok URL（每次重啟 ngrok 要更新一次）
BASE_URL = "https://gustoish-cuc-unvariegated.ngrok-free.dev"

client = OpenAI()

def search_incidents(question: str, limit: int = 5):
    """
    呼叫你自己的 /search API，用使用者的問題當 keyword 搜尋事故。
    """
    url = f"{BASE_URL}/search"
    try:
        resp = requests.get(url, params={"keyword": question, "limit": limit}, timeout=20)
        resp.raise_for_status()
    except Exception as e:
        print("查詢 API 發生錯誤：", e)
        return None

    data = resp.json()
    return data  # {'keyword': ..., 'count': ..., 'results': [...]}


def format_incidents_for_prompt(data: dict) -> str:
    """
    把 /search 回傳的 JSON 整理成給 LLM 看的文字。
    """
    if not data or data.get("count", 0) == 0:
        return "（查無相關事故案例。）"

    lines = []
    for i, item in enumerate(data["results"], start=1):
        lines.append(f"案例 {i}:")
        lines.append(f"- 行業別：{item.get('industry')}")
        lines.append(f"- 災害型態：{item.get('incident')}")
        lines.append(f"- 描述摘要：{item.get('description_summary')}")
        lines.append(f"- 原因摘要：{item.get('cause_summary')}")
        lines.append("")
    return "\n".join(lines)


def ask_osh_assistant(question: str, limit: int = 5, model: str = "gpt-4o-mini") -> str:
    """
    使用 OpenAI LLM：
    1. 先用 question 當 keyword 查你的職安事故 API
    2. 再請模型根據查到的案例 + 問題，給出分析與建議
    """
    incidents = search_incidents(question, limit=limit)
    incidents_text = format_incidents_for_prompt(incidents)

    system_prompt = """你是一位職業安全與健康的專家。
你會根據提供的事故案例資料，回答使用者的問題。
請：
1. 先用簡短中文回答問題的核心。
2. 再用條列整理：常見事故原因、預防措施。
3. 若查無相關案例，要直接說明「查無相關事故」，並改用一般職安原則回答。"""

    user_prompt = f"""使用者問題：
{question}

以下是從職安事故資料庫查到的相關案例（可能為 0 筆）：

{incidents_text}
"""

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.3,
    )

    answer = completion.choices[0].message.content
    return answer


In [ ]:
question = "有沒有高處墜落的案例？常見原因是什麼？怎麼預防？"
answer = ask_osh_assistant(question, limit=5)
print(answer)